# Enrichment Analysis

Enrichment analysis is used to found the pathway related to microbiome abundance and its metabolite profiles. 
"Function" usually refers to gene families such as KEGG orthologs and Enzyme Classification numbers, but predictions can be made for any arbitrary trait. 
Similarly, predictions are typically based on 16S rRNA gene sequencing data, but other marker genes can also be used.
We will use PICRUSt2 and visualized using ggpicrust2.

PICRUSt2 (Phylogenetic Investigation of Communities by Reconstruction of Unobserved States) is a software for predicting functional abundances based only on marker gene sequences. You can follow the installation and the tutorial from [PICRUSt2 GitHub](https://github.com/picrust/picrust2/wiki/PICRUSt2-Tutorial-(v2.5.2)).

The key steps of the PICRUSt2 pipeline are indicated in the below flowchart.



<img src="img/picrust2.png"/>

<center> General Workflow of PICRUSt2 Pipeline </center>

This enrichment analysis can analyzed for the statistically significant groups (Please refer to previous [statistics analysis](Statistics_Analysis.ipynb).
You need to provide some input data:
- metadata in tsv format
- microbiome sequence in fasta format 
- abundance table in biom format

First, we need to convert the microbiome sequence in fasta format from 'representative-sequence.qza' file

In [ ]:
qiime tools export --input-path [path]/rep-seqs.qza  --output-path [path]/

Then run the PICRUSt2 pipeline

In [ ]:
picrust2_pipeline.py -s [path]/dna-sequences.fasta -i [path]/asv-table.biom -o [path]/picrust2_out_pipeline -p 8

From the PICRUSt2 pipeline, it will generate some data, as seen in figure below:
<img src="img/picrust2_output.png"/>

<center> PICRUSt2 Output </center>

For PICRUSt2 results, we can the visualized it by using 'ggpicrust2 package' in Bioconductor. You can follow the installation instruction and tutorial from [here](https://github.com/cafferychen777/ggpicrust2).

Here is the ggpicrust2 workflow:
<img src="img/workflow.png"/>

<center> ggpicrust2 workflow </center>

We can analyzed the KEGG Orthology(KO) (classification system developed by the Kyoto Encyclopedia of Genes and Genomes (KEGG) data-base) and Metacyc pathway


The input file used for KO visualization is from 'KO_metagenome_out' directory --> 'pred_metagenome_unstrat.tsv', 
meanwhile the input file for Metacyc pathway is from 'pathways_out' directory --> 'path_abun_unstrat.tsv'
The input file will be looked like this below figure:
<img src="img/KO_abun.png"/>

<center> KO input data </center>

<img src="img/metacyc.png"/>

<center> Metacyc input data </center>

In [ ]:
library(readr)
library(MicrobiomeStat)
library(GGally)
library(ggpicrust2)
library(tibble)
library(tidyverse)
library(ggprism)
library(patchwork)

# Use the example data in ggpicrust2 package

# Analyze the KO data
data(ko_abundance)
data(metadata)
results_data_input <- ggpicrust2(data = ko_abundance,
                                 metadata = metadata,
                                 group = "Environment",
                                 pathway = "KO",
                                 daa_method = "LinDA",
                                 ko_to_kegg = TRUE,
                                 order = "pathway_class",
                                 p_values_bar = TRUE,
                                 x_lab = "pathway_name")
results_data_input[[1]]$plot
KO_results <- results_data_input[[1]]$results

# Perform pathway DAA using LinDA method
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
ko_daa_results_df <- pathway_daa(abundance = ko_abundance %>% column_to_rownames("#NAME"), metadata = metadata, group = "Environment", daa_method = "LinDA")

# Annotate MetaCyc pathway results without KO to KEGG conversion
ko_daa_annotated_results_df <- pathway_annotation(pathway = "KO", daa_results_df = ko_daa_results_df, ko_to_kegg = FALSE)

# Generate pathway heatmap
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
ko_feature_with_p_0.05 <- ko_daa_results_df %>% filter(p_adjust < 0.05) %>% top_n(10, p_adjust)
pathway_heatmap(abundance = ko_abundance %>% right_join(ko_feature_with_p_0.05 %>% select(all_of(c("feature"))),by = c("#NAME" = "feature")) %>% column_to_rownames("#NAME"), metadata = metadata, group = "Environment")

# Generate pathway PCA plot
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
pathway_pca(abundance = ko_abundance %>% column_to_rownames("#NAME"), metadata = metadata, group = "Environment")

# Run pathway DAA for multiple methods
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
methods <- c("ALDEx2", "DESeq2", "edgeR")
ko_daa_results_list <- lapply(methods, function(method) {
  pathway_daa(abundance = ko_abundance %>% column_to_rownames("#NAME"), metadata = metadata, group = "Environment", daa_method = method)
})

# Compare results across different methods
ko_comparison_results <- compare_daa_results(daa_results_list = ko_daa_results_list, method_names = c("ALDEx2_Welch's t test", "ALDEx2_Wilcoxon rank test", "DESeq2", "edgeR"))


#############################################################################

# Analyze the EC or MetaCyc pathway
data(metacyc_abundance)
results_file_input <- ggpicrust2(data = metacyc_abundance,
                                 metadata = metadata,
                                 group = "Environment",
                                 pathway = "MetaCyc",
                                 daa_method = "LinDA",
                                 ko_to_kegg = FALSE,
                                 order = "group",
                                 p_values_bar = TRUE,
                                 x_lab = "description")
results_file_input[[1]]$plot
metacyc_results <- results_file_input[[1]]$results


# Perform pathway DAA using LinDA method
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
metacyc_daa_results_df <- pathway_daa(abundance = metacyc_abundance %>% column_to_rownames("pathway"), metadata = metadata, group = "Environment", daa_method = "LinDA")

# Annotate MetaCyc pathway results without KO to KEGG conversion
metacyc_daa_annotated_results_df <- pathway_annotation(pathway = "MetaCyc", daa_results_df = metacyc_daa_results_df, ko_to_kegg = FALSE)

# Generate pathway heatmap
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
metacyc_feature_with_p_0.05 <- metacyc_daa_results_df %>% filter(p_adjust < 0.05) %>% top_n(10, p_adjust)
pathway_heatmap(abundance = metacyc_abundance %>% right_join(metacyc_feature_with_p_0.05 %>% select(all_of(c("feature"))),by = c("pathway" = "feature")) %>% column_to_rownames("pathway"), metadata = metadata, group = "Environment")

# Generate pathway PCA plot
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
pathway_pca(abundance = metacyc_abundance %>% column_to_rownames("pathway"), metadata = metadata, group = "Environment")

# Run pathway DAA for multiple methods
# Please change column_to_rownames() to the feature column if you are not using example dataset
# Please change group to "your_group_column" if you are not using example dataset
methods <- c("ALDEx2", "DESeq2", "edgeR")
metacyc_daa_results_list <- lapply(methods, function(method) {
  pathway_daa(abundance = metacyc_abundance %>% column_to_rownames("pathway"), metadata = metadata, group = "Environment", daa_method = method)
})

# Compare results across different methods
metacyc_comparison_results <- compare_daa_results(daa_results_list = metacyc_daa_results_list, method_names = c("ALDEx2_Welch's t test", "ALDEx2_Wilcoxon rank test", "DESeq2", "edgeR"))


The plots generated from above analysis are seen in below figure:

<img src="img/KO_results.png"/>

<center> KO results plot </center>

<img src="img/metacyc_results.png"/>

<center> Metacyc results plot </center>

Congratulation!! You've pass all the workshop!!
Thank you for joining our workshop. See you next event!!

Kind regards,
Bioinformatics Core Facilities - IMERI FKUI
------------------